**Pearson and Spearman Correlation**

Pearson measures the linear relationship between the two sets of values; Spearman measures the rank-based (monotonic) relationship.

In [ ]:
import pandas as pd
from scipy.stats import pearsonr, spearmanr

# Assume your data is in a CSV with 32 columns: 2 per gene
# Column naming: Gene1_FeatureA, Gene1_FeatureB, ..., Gene16_FeatureB
df = pd.read_csv("my file.csv")

# Flatten all values into single arrays
featureA_all = []
featureB_all = []

for i in range(1, 17):  # Assuming 16 genes
    a_col = f"Gene{i}_FeatureA"
    b_col = f"Gene{i}_FeatureB"
    featureA_all.extend(df[a_col].dropna().tolist())
    featureB_all.extend(df[b_col].dropna().tolist())

# Make sure both lists are same length
min_len = min(len(featureA_all), len(featureB_all))
featureA_all = featureA_all[:min_len]
featureB_all = featureB_all[:min_len]

# Correlation
print("Pearson:", pearsonr(featureA_all, featureB_all))
print("Spearman:", spearmanr(featureA_all, featureB_all))

**Next: Per-Gene Correlations → Meta-Analysis.** You can calculate the correlation between the two features per gene (i.e., correlation within each gene’s cloud), and then do a meta-correlation to assess overall tendency.

In [ ]:
gene_corrs = []

for i in range(1, 17):
    a_col = f"Gene{i}_FeatureA"
    b_col = f"Gene{i}_FeatureB"
    
    a_vals = df[a_col].dropna()
    b_vals = df[b_col].dropna()
    
    min_len = min(len(a_vals), len(b_vals))
    corr, _ = spearmanr(a_vals[:min_len], b_vals[:min_len])
    gene_corrs.append(corr)

# Now summarize the gene-level correlations
import numpy as np
print("Mean gene-level correlation:", np.mean(gene_corrs))

**Next: Significance levels for individual genes**

In [ ]:
import pandas as pd
from scipy.stats import ttest_ind

# Update to include Gene1 through Gene16
gene_names = [f"Gene{i}" for i in range(1, 17)]
results = []

for gene in gene_names:
    col_a = f"{gene}_FeatureA"
    col_b = f"{gene}_FeatureB"

    a_vals = df[col_a].dropna()
    b_vals = df[col_b].dropna()

    # Run t-test
    t_stat, p_val = ttest_ind(a_vals, b_vals, equal_var=False)

    # Calculate effect size (mean difference)
    mean_diff = a_vals.mean() - b_vals.mean()

    results.append({
        "Gene": gene,
        "FeatureA_Mean": a_vals.mean(),
        "FeatureB_Mean": b_vals.mean(),
        "Mean_Difference": mean_diff,
        "p_value": p_val
    })

# Create DataFrame
ttest_df = pd.DataFrame(results)

# Apply stricter significance threshold
ttest_df["Significant"] = ttest_df["p_value"] < 0.001

# Extract list of significant genes
significant_genes_list = ttest_df[ttest_df["Significant"]]["Gene"].tolist()

# Show the results
print("Significant Genes (p < 0.001):", significant_genes_list)
display(ttest_df.sort_values("p_value"))

**Next: Code for multi-metric heatmap**

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Recalculate log-transformed p-values
ttest_df["-log10(p_value)"] = -np.log10(ttest_df["p_value"])

# Sort genes by p-value
ttest_df = ttest_df.sort_values("-log10(p_value)", ascending=False)

# Build matrix for heatmap
heat_data = ttest_df.set_index("Gene")[["-log10(p_value)", "Mean_Difference"]].T

# Plot and capture figure
fig, ax = plt.subplots(figsize=(15, 3))
sns.heatmap(
    heat_data,
    cmap="vlag",
    annot=True,
    fmt=".2f",
    linewidths=0.5,
    cbar_kws={'label': 'Value'},
    ax=ax
)

# Labels
ax.set_title("Gene-wise Heatmap: Significance (-log10 p) and Mean Difference")
ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Layout and export
plt.tight_layout()
fig.savefig("gene_significance_heatmap.svg", format="svg")  # Save BEFORE plt.show()!!!
plt.show()

**Next: Step-by-Step Python Code for Permutation Testing**

In [ ]:
#Permutation testing

import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

# Replace this with your real DataFrame
df = pd.read_csv("250418_Lily Meta_CSV.csv")

# Simulated list of your 16 genes
genes = [f"Gene{i+1}" for i in range(16)]

# Set number of permutations
n_permutations = 10000

# Create an empty results list
results = []

for gene in genes:
    # Get the feature values, remove NaNs
    a_vals = df[f"{gene}_FeatureA"].dropna().values
    b_vals = df[f"{gene}_FeatureB"].dropna().values

    observed_diff = np.mean(a_vals) - np.mean(b_vals)
    combined = np.concatenate([a_vals, b_vals])
    n_A = len(a_vals)

    # Perform permutation
    perm_diffs = []
    for _ in range(n_permutations):
        shuffled = shuffle(combined, random_state=None)
        perm_diff = np.mean(shuffled[:n_A]) - np.mean(shuffled[n_A:])
        perm_diffs.append(perm_diff)

    # Compute two-tailed p-value
    perm_diffs = np.array(perm_diffs)
    p_val = np.mean(np.abs(perm_diffs) >= np.abs(observed_diff))

    results.append({
        "Gene": gene,
        "Observed_Diff": observed_diff,
        "Permutation_p_value": p_val
    })

# Create DataFrame of results
perm_results_df = pd.DataFrame(results)
perm_results_df.sort_values("Permutation_p_value", inplace=True)

# Add a readable label for very small p-values
perm_results_df["Permutation_p_value_label"] = perm_results_df["Permutation_p_value"].apply(
    lambda p: "<0.001" if p < 0.001 else f"{p:.6f}"
)

# Display top results
print(perm_results_df)

# Display with more precision
pd.set_option('display.float_format', '{:.10f}'.format)
print(perm_results_df)

# Optional: Plot histogram of permuted diffs for a gene
# Change gene index as needed (0–16)
gene_index = 0
gene_name = perm_results_df.iloc[gene_index]["Gene"]
a_vals = df[f"{gene_name}_FeatureA"].dropna().values
b_vals = df[f"{gene_name}_FeatureB"].dropna().values
combined = np.concatenate([a_vals, b_vals])
n_A = len(a_vals)

perm_diffs = [
    np.mean(shuffle(combined)[:n_A]) - np.mean(shuffle(combined)[n_A:])
    for _ in range(n_permutations)
]

plt.hist(perm_diffs, bins=30, alpha=0.7, label="Null distribution")
plt.axvline(np.mean(a_vals) - np.mean(b_vals), color='red', linestyle='--', label="Observed diff")
plt.title(f"Permutation Test for {gene_name}")
plt.xlabel("Mean difference (A - B)")
plt.ylabel("Frequency")
plt.legend()
plt.show()

**Print out individual plots using GridSpec By changing the value of gene_index all sixteen graphs can be printed individually.**

In [ ]:
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import numpy as np
​
# Select a gene to visualize (can change the index)
gene_index = 14
gene_name = perm_results_df.iloc[gene_index]["Gene"]
a_vals = df[f"{gene_name}_FeatureA"].dropna().values
b_vals = df[f"{gene_name}_FeatureB"].dropna().values
combined = np.concatenate([a_vals, b_vals])
n_A = len(a_vals)
observed_diff = np.mean(a_vals) - np.mean(b_vals)
​
# Generate permutation diffs
perm_diffs = [
    np.mean(shuffle(combined)[:n_A]) - np.mean(shuffle(combined)[n_A:])
    for _ in range(10000)  # use same number as in main analysis
]
​
# Plot
plt.figure(figsize=(6, 4))
plt.hist(perm_diffs, bins=30, alpha=0.7, color="skyblue", edgecolor="gray", label="Null distribution")
plt.axvline(observed_diff, color="red", linestyle="--", linewidth=2, label="Observed diff")
plt.title(f"Permutation Test for {gene_name}", fontsize=12)
plt.xlabel("Mean difference (FeatureA - FeatureB)")
plt.ylabel("Frequency")
plt.legend()
plt.tight_layout()
​
# Save as Illustrator-friendly SVG
plt.savefig("permutation_test_{}.svg".format(gene_name), format='svg')
plt.show()

 **Code: Add FDR correction to your existing permutation test**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests

# Load your dataset
df = pd.read_csv("250418_Lily Meta_CSV.csv")

# List of your 16 genes
genes = [f"Gene{i+1}" for i in range(16)]

# Number of permutations
n_permutations = 10000

# Store raw p-values and observed differences
results = []

for gene in genes:
    a_vals = df[f"{gene}_FeatureA"].dropna().values
    b_vals = df[f"{gene}_FeatureB"].dropna().values

    if len(a_vals) == 0 or len(b_vals) == 0:
        continue  # Skip if no data for this gene

    observed_diff = np.mean(a_vals) - np.mean(b_vals)
    combined = np.concatenate([a_vals, b_vals])
    n_A = len(a_vals)

    perm_diffs = []
    for _ in range(n_permutations):
        shuffled = shuffle(combined, random_state=None)
        perm_diff = np.mean(shuffled[:n_A]) - np.mean(shuffled[n_A:])
        perm_diffs.append(perm_diff)

    perm_diffs = np.array(perm_diffs)
    p_val = np.mean(np.abs(perm_diffs) >= np.abs(observed_diff))

    results.append({
        "Gene": gene,
        "Observed_Diff": observed_diff,
        "Permutation_p_value": p_val
    })

# Convert to DataFrame
perm_results_df = pd.DataFrame(results)

# FDR correction (Benjamini-Hochberg)
reject, pvals_corrected, _, _ = multipletests(perm_results_df["Permutation_p_value"], method='fdr_bh')
perm_results_df["FDR_corrected_p"] = pvals_corrected
perm_results_df["Significant_FDR_0.05"] = reject

# Sort and label p-values
perm_results_df.sort_values("Permutation_p_value", inplace=True)
perm_results_df["Permutation_p_value_label"] = perm_results_df["Permutation_p_value"].apply(
    lambda p: "<0.001" if p < 0.001 else f"{p:.6f}"
)
perm_results_df["FDR_corrected_p_label"] = perm_results_df["FDR_corrected_p"].apply(
    lambda p: "<0.001" if p < 0.001 else f"{p:.6f}"
)

# Display full table with precision
pd.set_option('display.float_format', '{:.10f}'.format)
print(perm_results_df)

**Step-by-step Visualization Code for FDR-Significant Genes:**

In [ ]:
from statsmodels.stats.multitest import multipletests

# Perform FDR correction
perm_results_df["FDR_p_value"] = multipletests(perm_results_df["Permutation_p_value"], method='fdr_bh')[1]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Optional: Add -log10 FDR p-value for plotting
perm_results_df["neg_log10_fdr"] = -np.log10(perm_results_df["FDR_p_value"].replace(0, 1e-10))

# Filter genes that passed FDR threshold
fdr_threshold = 0.05
signif_df = perm_results_df[perm_results_df["FDR_p_value"] < fdr_threshold].copy()
signif_df.sort_values("Observed_Diff", inplace=True)

# -------- Bar Plot -------- #
plt.figure(figsize=(10, 6))
sns.barplot(data=signif_df, x="Gene", y="Observed_Diff", palette="coolwarm")
plt.axhline(0, color='gray', linestyle='--')
plt.title("Observed Differences (FeatureA - FeatureB) for FDR-Significant Genes")
plt.ylabel("Observed Mean Difference")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("FDR_Significant_BarPlot.svg", format="svg")
plt.show()

# -------- Heatmap -------- #
heatmap_data = signif_df.set_index("Gene")[["Observed_Diff", "neg_log10_fdr"]]
heatmap_data.columns = ["Obs. Diff (A - B)", "-log10(FDR p)"]

plt.figure(figsize=(8, max(4, 0.5 * len(heatmap_data))))
sns.heatmap(heatmap_data, annot=True, cmap="vlag", center=0, fmt=".2f", cbar_kws={"label": "Value"})
plt.title("FDR-Significant Genes: Observed Diff & -log10(FDR p)")
plt.tight_layout()
plt.savefig("FDR_Significant_Heatmap.svg", format="svg")
plt.show()